In [15]:
%matplotlib inline
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import initializers as init
print(tf.__version__)


2.0.0


In [16]:
train_data = pd.read_csv('F:/AIdatabase/house-prices/train.csv')
test_data = pd.read_csv('F:/AIdatabase/house-prices/test.csv')

In [17]:
train_data.shape

(1460, 81)

In [18]:
test_data.shape

(1459, 80)

In [19]:
train_data.iloc[0:4,[0,1,2,3,-3,-2,-1]]

,Id,MSSubClass,MSZoning,LotFrontage,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,WD,Normal,208500
1,2,20,RL,80.0,WD,Normal,181500
2,3,60,RL,68.0,WD,Normal,223500
3,4,70,RL,60.0,WD,Abnorml,140000


In [20]:
all_features = pd.concat((train_data.iloc[:,1:-1],test_data.iloc[:,1:]))


In [21]:
numeric_features = all_features.dtypes[all_features.dtypes != 'object'].index
all_features[numeric_features] = all_features[numeric_features].apply(
    lambda x:(x - x.mean())/(x.std()))
all_features = all_features.fillna(0)

In [22]:
all_features = pd.get_dummies(all_features,dummy_na=True)
all_features.shape

(2919, 354)

In [23]:
n_train = train_data.shape[0]
train_features = np.array(all_features[:n_train].values,dtype=np.float)
test_features = np.array(all_features[n_train:].values,dtype=np.float)
train_labels = np.array(train_data.SalePrice.values.reshape(-1,1),dtype=np.float)

In [24]:
def get_net():
    net = keras.models.Sequential()
    net.add(keras.layers.Dense(1))
    return net

In [25]:
log_rmse = tf.keras.losses.mean_squared_logarithmic_error


In [39]:
def get_k_fold_data(k,i,X,y):
    assert k>1
    fold_size = X.shape[0]
    X_train,y_train = None,None
    for j in range(k):
        idx = slice(j * fold_size,(j + 1) * fold_size)
        X_part,y_part = X[idx,:],y[idx]
        if j == i:
            X_valid,y_valid = X_part,y_part
        elif X_train is None:
            X_train,y_train = X_part, y_part
        else:
            X_train = tf.concat([X_train,X_part],axis=0)
            y_train = tf.concat([y_train,y_part],axis=0)
    return X_train,y_train,X_valid,y_valid

In [40]:
def k_fold(k, X_train, y_train, num_epochs,
           learning_rate, weight_decay, batch_size):
    train_l_sum, valid_l_sum = 0, 0
    for i in range(k):
        # create model
        data = get_k_fold_data(k, i, X_train, y_train)
        net=get_net()
        # Compile model
        net.compile(loss=tf.keras.losses.mean_squared_logarithmic_error, optimizer=tf.keras.optimizers.Adam(learning_rate))
        # Fit the model
        history=net.fit(data[0], data[1],validation_data=(data[2], data[3]), epochs=num_epochs, batch_size=batch_size,validation_freq=1,verbose=0)
        loss = history.history['loss']
        val_loss = history.history['val_loss']
        print('fold %d, train rmse %f, valid rmse %f'
              % (i, loss[-1], val_loss[-1]))
    plt.subplot(1, 2, 2)
    plt.plot(loss, label='train')
    plt.plot(val_loss, label='valid')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.show()


In [ ]:
k, num_epochs, lr, weight_decay, batch_size = 5, 10, 5, 0, 64
k_fold(k, train_features, train_labels, num_epochs,lr, weight_decay, batch_size)
